In [18]:
import pathlib
import typing as tp

T = tp.TypeVar("T")

def read_sudoku(path: tp.Union[str, pathlib.Path]) -> tp.List[tp.List[str]]:
    """ Прочитать Судоку из указанного файла """
    path = pathlib.Path(path)
    with path.open() as f:
        puzzle = f.read()
    return create_grid(puzzle)


def create_grid(puzzle: str) -> tp.List[tp.List[str]]:
    digits = [c for c in puzzle if c in "123456789."]
    grid = group(digits, 9)
    return grid


def display(grid: tp.List[tp.List[str]]) -> None:
    """Вывод Судоку """
    width = 2
    line = "+".join(["-" * (width * 3)] * 3)
    for row in range(9):
        print(
            "".join(
                grid[row][col].center(width) + ("|" if str(col) in "25" else "") for col in range(9)
            )
        )
        if str(row) in "25":
            print(line)
    print()


def group(values: tp.List[T], n: int) -> tp.List[tp.List[T]]:
    """
    Сгруппировать значения values в список, состоящий из списков по n элементов
    >>> group([1,2,3,4], 2)
    [[1, 2], [3, 4]]
    >>> group([1,2,3,4,5,6,7,8,9], 3)
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
    """
    new_group = list()
    for i in range(0, len(values), n):
        new_group.append(values[i:(i + n)])
    return new_group


def get_row(grid: tp.List[tp.List[str]], pos: tp.Tuple[int, int]) -> tp.List[str]:
    """Возвращает все значения для номера строки, указанной в pos
    >>> get_row([['1', '2', '.'], ['4', '5', '6'], ['7', '8', '9']], (0, 0))
    ['1', '2', '.']
    >>> get_row([['1', '2', '3'], ['4', '.', '6'], ['7', '8', '9']], (1, 0))
    ['4', '.', '6']
    >>> get_row([['1', '2', '3'], ['4', '5', '6'], ['.', '8', '9']], (2, 0))
    ['.', '8', '9']
    """
    return grid[pos[0]]


def get_col(grid: tp.List[tp.List[str]], pos: tp.Tuple[int, int]) -> tp.List[str]:
    """Возвращает все значения для номера столбца, указанного в pos
    >>> get_col([['1', '2', '.'], ['4', '5', '6'], ['7', '8', '9']], (0, 0))
    ['1', '4', '7']
    >>> get_col([['1', '2', '3'], ['4', '.', '6'], ['7', '8', '9']], (0, 1))
    ['2', '.', '8']
    >>> get_col([['1', '2', '3'], ['4', '5', '6'], ['.', '8', '9']], (0, 2))
    ['3', '6', '9']
    """
    col = list()
    for i in grid:
        col.append(i[pos[1]])
    return col


def get_block(grid: tp.List[tp.List[str]], pos: tp.Tuple[int, int]) -> tp.List[str]:
    """Возвращает все значения из квадрата, в который попадает позиция pos
    >>> grid = read_sudoku('puzzle1.txt')
    >>> get_block(grid, (0, 1))
    ['5', '3', '.', '6', '.', '.', '.', '9', '8']
    >>> get_block(grid, (4, 7))
    ['.', '.', '3', '.', '.', '1', '.', '.', '6']
    >>> get_block(grid, (8, 8))
    ['2', '8', '.', '.', '.', '5', '.', '7', '9']
    """
    block = list()
    line = (pos[0]) // 3
    collumn = (pos[1]) // 3
    for i in range(line*3, line*3 + 3):
        block.extend(grid[i][3*collumn:3*collumn+3])
    return block


def find_empty_positions(grid: tp.List[tp.List[str]]) -> tp.Optional[tp.Tuple[int, int]]:
    """Найти первую свободную позицию в пазле
    >>> find_empty_positions([['1', '2', '.'], ['4', '5', '6'], ['7', '8', '9']])
    (0, 2)
    >>> find_empty_positions([['1', '2', '3'], ['4', '.', '6'], ['7', '8', '9']])
    (1, 1)
    >>> find_empty_positions([['1', '2', '3'], ['4', '5', '6'], ['.', '8', '9']])
    (2, 0)
    """
    ind1 = -1;
    ind2 = -1;
    for i in range(len(grid)):
        if '.' in grid[i]:
            ind1 = i;
            ind2 = grid[i].index('.')
            break
    return (ind1, ind2)


def find_possible_values(grid: tp.List[tp.List[str]], pos: tp.Tuple[int, int]) -> tp.Set[str]:
    """Вернуть множество возможных значения для указанной позиции
    >>> grid = read_sudoku('puzzle1.txt')
    >>> values = find_possible_values(grid, (0,2))
    >>> values == {'1', '2', '4'}
    True
    >>> values = find_possible_values(grid, (4,7))
    >>> values == {'2', '5', '9'}
    True
    """
    a = get_row(grid, pos)
    b = get_col(grid, pos)
    c = get_block(grid, pos)
    values = set(a + b + c)
    return {'1','2','3','4','5','6','7','8','9','.'}.difference(values)


def solve(grid: tp.List[tp.List[str]]) -> tp.Optional[tp.List[tp.List[str]]]:
    """ Решение пазла, заданного в grid """
    """ Как решать Судоку?
        1. Найти свободную позицию
        2. Найти все возможные значения, которые могут находиться на этой позиции
        3. Для каждого возможного значения:
            3.1. Поместить это значение на эту позицию
            3.2. Продолжить решать оставшуюся часть пазла
    >>> grid = read_sudoku('puzzle1.txt')
    >>> solve(grid)
    [['5', '3', '4', '6', '7', '8', '9', '1', '2'], ['6', '7', '2', '1', '9', '5', '3', '4', '8'], ['1', '9', '8', '3', '4', '2', '5', '6', '7'], ['8', '5', '9', '7', '6', '1', '4', '2', '3'], ['4', '2', '6', '8', '5', '3', '7', '9', '1'], ['7', '1', '3', '9', '2', '4', '8', '5', '6'], ['9', '6', '1', '5', '3', '7', '2', '8', '4'], ['2', '8', '7', '4', '1', '9', '6', '3', '5'], ['3', '4', '5', '2', '8', '6', '1', '7', '9']]
    """
    pos = find_empty_positions(grid)
    if pos == (-1, -1):
        return grid
    pos_values = sorted(list(find_possible_values(grid, pos)))
    if pos_values:
        for guess in pos_values:
            grid[pos[0]][pos[1]] = guess
            if solve(grid):
                return grid
            grid[pos[0]][pos[1]] = '.'
    return False


def check_solution(solution: tp.List[tp.List[str]]) -> bool:
    """ Если решение solution верно, то вернуть True, в противном случае False """
    # TODO: Add doctests with bad puzzles
    check_set = {'1','2','3','4','5','6','7','8','9'}
    for i in range(9):
        for j in range(9):
            a = set(get_row(solution, (i, j)))
            b = set(get_col(solution, (i, j)))
            c = set(get_block(solution, (i, j)))
            if (check_set.difference(a)) or (check_set.difference(b)) or (check_set.difference(c)):
                return False
    return True


def generate_sudoku(N: int) -> tp.List[tp.List[str]]:
    """Генерация судоку заполненного на N элементов
    >>> grid = generate_sudoku(40)
    >>> sum(1 for row in grid for e in row if e == '.')
    41
    >>> solution = solve(grid)
    >>> check_solution(solution)
    True
    >>> grid = generate_sudoku(1000)
    >>> sum(1 for row in grid for e in row if e == '.')
    0
    >>> solution = solve(grid)
    >>> check_solution(solution)
    True
    >>> grid = generate_sudoku(0)
    >>> sum(1 for row in grid for e in row if e == '.')
    81
    >>> solution = solve(grid)
    >>> check_solution(solution)
    True
    """
    import random as rd
    grid_template = [['1', '2', '3', '4', '5', '6', '7', '8', '9'],
                     ['4', '5', '6', '7', '8', '9', '1', '2', '3'],
                     ['7', '8', '9', '1', '2', '3', '4', '5', '6'],
                     ['2', '3', '4', '5', '6', '7', '8', '9', '1'],
                     ['5', '6', '7', '8', '9', '1', '2', '3', '4'],
                     ['8', '9', '1', '2', '3', '4', '5', '6', '7'],
                     ['3', '4', '5', '6', '7', '8', '9', '1', '2'],
                     ['6', '7', '8', '9', '1', '2', '3', '4', '5'],
                     ['9', '1', '2', '3', '4', '5', '6', '7', '8']]
    grid_1 = [['.'] * 9 for i in range(9)]
    grid_2 = [['.'] * 9 for i in range(9)]
    while not check_solution(grid_2):
        number_list1 = [i for i in range(9)]
        number_list2 = [i for i in range(9)]
        rd.shuffle(number_list1)
        for i in range(9):
            grid_1[i] = grid_template[number_list1[i]]
        rd.shuffle(number_list2)
        for i in range(9):
            for j in range(9):
                grid_2[j][i] = grid_1[j][number_list2[i]]
        grid_1 = [['.'] * 9 for i in range(9)]
    M = max(0, 81 - N)
    kol = 0;
    while kol != M:
        x = rd.randint(0,8)
        y = rd.randint(0,8)
        if grid_2[x][y] != '.':
            grid_2[x][y] = '.'
            kol += 1
    return grid_2


if __name__ == "__main__":
    for fname in ["puzzle1.txt", "puzzle2.txt", "puzzle3.txt"]:
        grid = read_sudoku(fname)
        display(grid)
        solution = solve(grid)
        if not solution:
            print(f"Puzzle {fname} can't be solved")
        else:
            display(solution)

5 3 . |. 7 . |. . . 
6 . . |1 9 5 |. . . 
. 9 8 |. . . |. 6 . 
------+------+------
8 . . |. 6 . |. . 3 
4 . . |8 . 3 |. . 1 
7 . . |. 2 . |. . 6 
------+------+------
. 6 . |. . . |2 8 . 
. . . |4 1 9 |. . 5 
. . . |. 8 . |. 7 9 

5 3 4 |6 7 8 |9 1 2 
6 7 2 |1 9 5 |3 4 8 
1 9 8 |3 4 2 |5 6 7 
------+------+------
8 5 9 |7 6 1 |4 2 3 
4 2 6 |8 5 3 |7 9 1 
7 1 3 |9 2 4 |8 5 6 
------+------+------
9 6 1 |5 3 7 |2 8 4 
2 8 7 |4 1 9 |6 3 5 
3 4 5 |2 8 6 |1 7 9 

. 9 4 |. . . |1 3 . 
. . . |. . . |. . . 
. . . |. 7 6 |. . 2 
------+------+------
. 8 . |. 1 . |. . . 
. 3 2 |. . . |. . . 
. . . |2 . . |. 6 . 
------+------+------
. . . |. 5 . |4 . . 
. . . |. . 8 |. . 7 
. . 6 |3 . 4 |. . 8 

7 9 4 |5 8 2 |1 3 6 
2 6 8 |9 3 1 |7 4 5 
3 1 5 |4 7 6 |9 8 2 
------+------+------
6 8 9 |7 1 5 |3 2 4 
4 3 2 |8 6 9 |5 7 1 
1 5 7 |2 4 3 |8 6 9 
------+------+------
8 2 1 |6 5 7 |4 9 3 
9 4 3 |1 2 8 |6 5 7 
5 7 6 |3 9 4 |2 1 8 

8 . . |4 . 6 |. . 7 
. . . |. . . |4 . . 
. 1 . |. . . |6 5 . 
------+--

In [16]:
read_sudoku('puzzle1.txt')

[['5', '3', '.', '.', '7', '.', '.', '.', '.'],
 ['6', '.', '.', '1', '9', '5', '.', '.', '.'],
 ['.', '9', '8', '.', '.', '.', '.', '6', '.'],
 ['8', '.', '.', '.', '6', '.', '.', '.', '3'],
 ['4', '.', '.', '8', '.', '3', '.', '.', '1'],
 ['7', '.', '.', '.', '2', '.', '.', '.', '6'],
 ['.', '6', '.', '.', '.', '.', '2', '8', '.'],
 ['.', '.', '.', '4', '1', '9', '.', '.', '5'],
 ['.', '.', '.', '.', '8', '.', '.', '7', '9']]